<h1>contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import dataiku
import numpy as np
from path import Path
import oct2py as op
import pandas as pd

In [2]:
Path("octave-workspace").remove_p()
Path("Detector.mat").remove_p()
Path("Log.txt").remove_p()

Path('Log.txt')

In [3]:
model_name = 'one_tree_1_img'

In [4]:
octave_code = 'octave'

Compile **C++** 'MEX' files for Octave/Matlab for local OS - might need some massaging depending on local OS

In [5]:
%%capture
"""
!cd {octave_code}/toolbox/channels/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE rgbConvertMex.cpp
!cd {octave_code}/toolbox/channels/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE gradientMex.cpp
!cd {octave_code}/toolbox/channels/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE convConst.cpp
!cd {octave_code}/toolbox/channels/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE imPadMex.cpp
!cd {octave_code}/toolbox/channels/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE imResampleMex.cpp
!cd {octave_code}/toolbox/classify/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE binaryTreeTrain1.cpp
!cd {octave_code}/toolbox/classify/private/ ; mkoctfile --mex -DMATLAB_MEX_FILE forestInds.cpp
"""

In [6]:
octave = op.Oct2Py()
octave.restart()
octave.eval('clear all')

In [7]:
%%capture
octave.addpath(octave.genpath(octave_code))

In [8]:
posWinDir = 'images/simple/posWinDir'
negWinDir = 'images/simple/negWinDir'

In [9]:
params = pd.read_csv('train_params.csv')

In [10]:
# params = train_params_df.loc[train_params_df['name'] == model_name]

opts_modelDs_height    = params['opts.modelDs.height'].values[0]
opts_modelDs_width     = params['opts.modelDs.width'].values[0]
opts_modelDsPad_height = params['opts.modelDsPad.height'].values[0]
opts_modelDsPad_width  = params['opts.modelDsPad.width'].values[0]
stride                 = params['stride'].values[0]
cascThr                = params['cascThr'].values[0]
cascCal                = params['cascCal'].values[0]
nWeak                  = params['nWeak'].values[0]
seed                   = params['seed'].values[0]
nPos                   = params['nPos'].values[0]
nNeg                   = params['nNeg'].values[0]
nPerNeg                = params['nPerNeg'].values[0]
nAccNeg                = params['nAccNeg'].values[0]
winsSave               = params['winsSave'].values[0]

In [12]:
%%capture
"""
opts_modelDs_height    = 19
opts_modelDs_width     = 19
opts_modelDsPad_height = 20
opts_modelDsPad_width  = 20
stride                 = 1
cascThr                = -1
cascCal                = 0.005
nWeak                  = 1
seed                   = 0
nPos                   = 1
nNeg                   = 0
nPerNeg                = 1
nAccNeg                = 1
winsSave               = 0
"""

In [11]:
Path("Detector.mat").remove_p()
Path("Log.txt").remove_p()
octave.eval("opts=acfTrain();")
octave.eval("opts.posWinDir = '" + posWinDir + "';")
octave.eval("opts.negWinDir = '" + negWinDir + "';")

In [18]:
octave.eval("opts.modelDs = [" + str(opts_modelDs_height) + "  " + str(opts_modelDs_width) + "];")
octave.eval("opts.modelDsPad = [" + str(opts_modelDsPad_height) + "  " + str(opts_modelDsPad_width) + "];")
octave.eval("opts.stride = " + str(stride) + ";")
octave.eval("opts.cascThr = " + str(cascThr) + ";")
octave.eval("opts.cascCal = " + str(cascCal) + ";")
octave.eval("opts.nWeak = " + str(nWeak) + ";")
octave.eval("opts.seed = " + str(seed) + ";")
octave.eval("opts.nPos = " + str(nPos) + ";")
octave.eval("opts.nNeg = " + str(nNeg) + ";")
octave.eval("opts.nPerNeg = " + str(nPerNeg) + ";")
octave.eval("opts.nAccNeg = " + str(nAccNeg) + ";")
octave.eval("opts.winsSave = " + str(winsSave) + ";")

octave.eval("opts.nWeak = " + str(1) + ";")

In [19]:
!ls {posWinDir}

area.01.png     cubic.01.png    lanczos4.01.png linear.01.png   nearest.01.png
area.02.png     cubic.02.png    lanczos4.02.png linear.02.png   nearest.02.png
area.03.png     cubic.03.png    lanczos4.03.png linear.03.png   nearest.03.png
area.04.png     cubic.04.png    lanczos4.04.png linear.04.png   nearest.04.png
area.05.png     cubic.05.png    lanczos4.05.png linear.05.png   nearest.05.png
area.06.png     cubic.06.png    lanczos4.06.png linear.06.png   nearest.06.png
area.07.png     cubic.07.png    lanczos4.07.png linear.07.png   nearest.07.png
area.08.png     cubic.08.png    lanczos4.08.png linear.08.png   nearest.08.png
area.09.png     cubic.09.png    lanczos4.09.png linear.09.png   nearest.09.png
area.10.png     cubic.10.png    lanczos4.10.png linear.10.png   nearest.10.png
area.11.png     cubic.11.png    lanczos4.11.png linear.11.png   nearest.11.png
area.12.png     cubic.12.png    lanczos4.12.png linear.12.png   nearest.12.png
area.13.png     cubic.13.png    lanczos4.13.png line

In [20]:
!ls {negWinDir}

area.GSD.png     lanczos4.GSD.png nearest.GSD.png
cubic.GSD.png    linear.GSD.png


In [21]:
Path("Detector.mat").remove_p()
Path("Log.txt").remove_p()
octave.eval("model = acfTrain(opts);")

---------------------------------------------------------------------------
Training stage 0
Done sampling windows (time=0s).
Computing lambdas... done (time=0s).
Extracting features... done (time=0s).
Done sampling windows (time=0s).
Extracting features... done (time=0s).
Training AdaBoost: nWeak=  1 nFtrs=250 pos=88 neg=3
Done training err=0.0000 fp=0.0000 fn=0.0000 (t=0.0s).
Done training stage 0 (time=1s).
---------------------------------------------------------------------------
Done training (time=1s).


Convert **Matlab** classifier object to JSON

In [24]:
octave.eval("simple_model_json = savejson('" + model_name + "',model);")
octave.eval("json_file = fopen('" + model_name + ".json" + "', 'w');")
octave.eval("fdisp(json_file, simple_model_json);")
octave.eval("fclose(json_file);")

0.0

In [27]:
!echo {model_name}

one_tree_1_img


In [28]:
!ls {model_name}.json

one_tree_1_img.json


In [29]:
!cat {model_name}.json

{
	"one_tree_1_img": {
		"opts": {
			"pPyramid": {
				"pChns": {
					"shrink": 4,
					"pColor": {
						"enabled": 1,
						"smooth": 1,
						"colorSpace": "luv"
					},
					"pGradMag": {
						"enabled": 1,
						"colorChn": 0,
						"normRad": 5,
						"normConst": 0.005,
						"full": 0
					},
					"pGradHist": {
						"enabled": 1,
						"binSize": null,
						"nOrients": 6,
						"softBin": 0,
						"useHog": 0,
						"clipHog": 0.2
					},
					"pCustom": [],
					"complete": 1
				},
				"nPerOct": 8,
				"nOctUp": 0,
				"nApprox": 7,
				"lambdas": [-0,0.0011,0.0011],
				"pad": [4,4],
				"minDs": [19,19],
				"smooth": 1,
				"concat": 1,
				"complete": 1
			},
			"filters": null,
			"modelDs": [19,19],
			"modelDsPad": [20,20],
			"pNms": {
				"type": "maxg",
				"overlap": 0.65,
				"ovrDnm": "min"
			},
			"stride": 4,
			"cascThr": -1,
			"cascCal": 0.005,
			"nWeak": 1,
			"pBoost": {
				"pTree": {
					"nBins": 256,
					"maxDepth": 2,
					"minWeight": 0.01,

In [0]:
Path("octave-workspace").remove_p()
Path("Detector.mat").remove_p()
Path("Log.txt").remove_p()

In [0]:
# !ls {train_model_JSON}

In [0]:
# !head -10 {train_model_JSON}/{model_name}.json